# Using XGboost for Variable Importance
In this notebook, the `xgboost` library will be used to determine the importance of the many technical indicators we collected in order to conduct feature reduction.

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb

In [19]:
df = pd.read_csv("data/fulldata.csv", index_col=0)
df.head()

,Date,open_aapl,high_aapl,low_aapl,close_aapl,volume_aapl,open_googl,high_googl,low_googl,close_googl,...,ema,macd,momentum,rsi_5,rsi_15,rsi_ratio,tr,atr,roc,vader_polarity
0,2013-05-09,16.421786,16.535714,16.270714,16.313214,398749904,21.792098,22.012812,21.726785,21.808114,...,16.373799,0.329583,15.313214,66.171526,64.804242,1.021099,0.295000,0.419158,0.165081,0.131248
1,2013-05-10,16.356071,16.418214,16.088572,16.177500,334849928,21.903956,22.034833,21.825130,22.027075,...,16.242933,0.325128,15.177500,57.572386,61.822287,0.931256,0.329643,0.412934,0.159885,0.146785
2,2013-05-13,16.125357,16.353572,16.125000,16.240714,317109240,21.993543,22.083127,21.855684,21.959510,...,16.241454,0.322504,15.240714,60.556778,62.679325,0.966136,0.228572,0.404487,0.140643,0.063889
3,2013-05-14,16.208928,16.257143,15.791071,15.852143,447115760,21.958759,22.238780,21.949751,22.198992,...,15.981913,0.288120,14.852143,39.310535,54.606048,0.719893,0.466071,0.379742,0.092901,0.049072
4,2013-05-15,15.684286,15.750000,15.084286,15.316071,741612360,22.409198,22.931701,22.372159,22.919439,...,15.538019,0.218220,14.316071,24.492002,45.872465,0.533915,0.767857,0.412962,0.057682,0.011088


In [20]:
df.shape

(2492, 51)

In [21]:
x = df.loc[:, df.columns != 'close_aapl']
x

,Date,open_aapl,high_aapl,low_aapl,volume_aapl,open_googl,high_googl,low_googl,close_googl,volume_googl,...,ema,macd,momentum,rsi_5,rsi_15,rsi_ratio,tr,atr,roc,vader_polarity
0,2013-05-09,16.421786,16.535714,16.270714,398749904,21.792098,22.012812,21.726785,21.808114,87945080,...,16.373799,0.329583,15.313214,66.171526,64.804242,1.021099,0.295000,0.419158,0.165081,0.131248
1,2013-05-10,16.356071,16.418214,16.088572,334849928,21.903956,22.034833,21.825130,22.027075,75955060,...,16.242933,0.325128,15.177500,57.572386,61.822287,0.931256,0.329643,0.412934,0.159885,0.146785
2,2013-05-13,16.125357,16.353572,16.125000,317109240,21.993543,22.083127,21.855684,21.959510,57893080,...,16.241454,0.322504,15.240714,60.556778,62.679325,0.966136,0.228572,0.404487,0.140643,0.063889
3,2013-05-14,16.208928,16.257143,15.791071,447115760,21.958759,22.238780,21.949751,22.198992,63190600,...,15.981913,0.288120,14.852143,39.310535,54.606048,0.719893,0.466071,0.379742,0.092901,0.049072
4,2013-05-15,15.684286,15.750000,15.084286,741612360,22.409198,22.931701,22.372159,22.919439,159658020,...,15.538019,0.218220,14.316071,24.492002,45.872465,0.533915,0.767857,0.412962,0.057682,0.011088
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2487,2023-03-27,159.940000,160.770000,157.870000,52390266,104.615000,104.760000,101.927300,102.460000,31120864,...,158.756022,2.923728,157.280000,58.221301,60.577495,0.961104,2.900000,3.498250,0.028928,0.154446
2488,2023-03-28,157.970000,158.490000,155.980000,45992152,102.440000,102.450000,99.740000,101.030000,32057865,...,158.018674,2.811818,156.650000,53.185052,59.113390,0.899712,2.510000,3.543964,0.039908,0.171291
2489,2023-03-29,159.370000,161.050000,159.350000,51305691,102.280000,102.490000,100.650000,101.390000,28779572,...,159.852891,2.940983,159.770000,69.511345,63.760854,1.090188,3.400000,3.478964,0.051678,0.149073
2490,2023-03-30,161.530000,162.470000,161.271000,49501689,100.910000,101.155000,99.780000,100.890000,33086183,...,161.524297,3.135504,161.360000,75.053359,65.878571,1.139268,1.700000,3.362507,0.078159,0.000000


In [29]:
y = df.close_aapl
y

0        16.313214
1        16.177500
2        16.240714
3        15.852143
4        15.316071
           ...    
2487    158.280000
2488    157.650000
2489    160.770000
2490    162.360000
2491    164.900000
Name: close_aapl, Length: 2492, dtype: float64

In [31]:
model = xgb.XGBRegressor
model.fit(x, y)

TypeError: fit() missing 1 required positional argument: 'y'